In [1]:
from lunax.data_processing.utils import *
from lunax.models import xgb_reg
from lunax.hyper_opt import OptunaTuner

In [2]:
df_train = load_data('./example_data/regression/train.csv', 'csv')
df_train

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [3]:
target = 'Listening_Time_minutes'

In [4]:
df_train = preprocess_data(df_train, target)

In [5]:
X_train, X_val, y_train, y_val = split_data(df_train, target)

In [6]:
# 创建调参器 使用默认搜索空间
tuner = OptunaTuner(
    n_trials=3,
    model_class="XGBRegressor",
)

In [7]:
# 执行优化
results = tuner.optimize(X_train, y_train, X_val, y_val)
# 获取最优参数
best_params = results['best_params']

[I 2025-05-18 10:32:29,817] A new study created in memory with name: no-name-336f23d0-c462-4760-a48e-7a9964abca9b
[I 2025-05-18 10:32:33,106] Trial 0 finished with value: 735.8300406602353 and parameters: {'booster': 'gblinear', 'seed': 0, 'eta': 0.1135122708349409, 'lambda': 86.38389774970184, 'reg_lambda': 0.9347634762640673, 'reg_alpha': 131}. Best is trial 0 with value: 735.8300406602353.
[I 2025-05-18 10:32:41,516] Trial 1 finished with value: 168.61212802074363 and parameters: {'booster': 'gbtree', 'seed': 0, 'eta': 0.11079429153624905, 'lambda': 69.35356931814624, 'reg_lambda': 0.1160963706726097, 'reg_alpha': 148, 'gamma': 6.6479062926789405, 'subsample': 0.9440451844603724, 'max_depth': 9, 'colsample_bytree': 0.8538933664508606, 'min_child_weight': 8, 'grow_policy': 'lossguide'}. Best is trial 1 with value: 168.61212802074363.
[I 2025-05-18 10:32:45,062] Trial 2 finished with value: 172.7194592863224 and parameters: {'booster': 'gbtree', 'seed': 0, 'eta': 0.10564427690522683, 

In [8]:
xgb_reg = xgb_reg(best_params)
xgb_reg.fit(X=X_train, y=y_train,k_fold=5)

[lunax]> Fold 1/5 - MSE: 168.7783, R2: 0.7708
[lunax]> Fold 2/5 - MSE: 170.6959, R2: 0.7689
[lunax]> Fold 3/5 - MSE: 168.8611, R2: 0.7712
[lunax]> Fold 4/5 - MSE: 171.2129, R2: 0.7668
[lunax]> Fold 5/5 - MSE: 170.7203, R2: 0.7682
[lunax]> Average scores - MSE: 170.0537, R2: 0.7691


In [9]:
xgb_reg.evaluate(X_val, y_val)

[lunax]> target value description:
+-------+--------+--------+-------+----------+
|   min |    max |   mean |   std |   median |
+=======+========+========+=======+==========+
|     0 | 119.97 |   45.4 | 27.13 |    43.36 |
+-------+--------+--------+-------+----------+
[lunax]> model evaluation results:
+-----------+--------+--------+-------+------+
| metrics   |   rmse |    mse |   mae |   r2 |
+===========+========+========+=======+======+
| values    |  13.01 | 169.23 |  9.46 | 0.77 |
+-----------+--------+--------+-------+------+


{'rmse': 13.008697591693954,
 'mse': 169.2262130321441,
 'mae': 9.464003618782165,
 'r2': 0.7700191270616175}